### Install required modules

In [ ]:
!pip install pyabsa --quiet --root-user-action=ignore 2> /dev/null
!pip install transformers==4.29.0 --quiet --root-user-action=ignore 2> /dev/null


### Import Required Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from pyabsa import ATEPCCheckpointManager
from pyabsa import available_checkpoints
from flask import Flask, request, jsonify
import re 
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

### Read in csv

In [ ]:
url = 'https://raw.githubusercontent.com/morayod/M_S_C_D_S/main/reviews.csv'
data = pd.read_csv(url)
data.head(10)

### Drop unused columns

In [ ]:
data = data.drop(['Time_submitted','Total_thumbsup','Reply'],axis=1)

In [ ]:
data.head()

### Review and clean data

In [ ]:
data['Rating'].value_counts()

In [ ]:
STOPWORDS = stopwords.words("english")

In [ ]:
def clean(x):
    x =  x.lower()
    x  =  re.sub("[^\w\d]"," ",x)
    x = " ".join(t for t in x.split() if t not in STOPWORDS)
    return x

In [ ]:
data['Review'] = data['Review'].apply(lambda x : clean(x))

In [ ]:
data.head()

In [ ]:
data['Review'][0:10]

In [ ]:
ax = sns.countplot(x="Rating", data=data)
for i in ax.containers:
    ax.bar_label(i,)

In [ ]:
text =list(data['Review'].values)

In [ ]:
text = list(data['Review'][0:100])

In [ ]:
text

## Explicit Aspect-based Sentiment Analysis Part

In [ ]:
checkpoint_map = available_checkpoints()

In [ ]:
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english',
                                   auto_device=True 
                                   )

In [ ]:
atepc_result = aspect_extractor.extract_aspect(inference_source=text,  #
                          pred_sentiment=True,
                          )

In [ ]:
atepc_result[0:5]

In [ ]:
datafile = 'https://raw.githubusercontent.com/morayod/M_S_C_D_S/main/randomly%20shuffled_reviews_only.csv'
test_df = pd.read_csv(datafile, encoding="utf-8")
test_df_100 = test_df.head(100)
test_data =list(test_df_100['Review'].values)
test_data = list(test_df_100['Review'][0:100])

atepc_result = aspect_extractor.extract_aspect(inference_source=test_data,  #
                          pred_sentiment=True,
                          )